Path

In [1]:
import os
import shutil

PATH_PROJECT_ROOT = "./"
os.chdir(PATH_PROJECT_ROOT)

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

2024-01-21 23:49:28.948036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 23:49:30.686103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.13.0


2024-01-21 23:49:33.681884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 23:49:33.952554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 23:49:33.952678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [40]:
MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz"
PRETRAINED_MODEL_NAME = "ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8"
CUSTOM_MODEL_NAME = "ssd_mobilenet_fpn_640x640"

PATHS = {
    "DATASET":os.path.join(".", "datasets", "v1"),
    "MODELS": os.path.join(".", "models"),
    "PRETRAINED_MODEL": os.path.join(".", "models", PRETRAINED_MODEL_NAME),
    "APIMODEL": os.path.join(".", "apimodel"),
    "PROTOC": os.path.join(".", "protoc"),
    "CUSTOM_MODELS": os.path.join(".", "custom_models"),
    "CUSTOM_MODEL": os.path.join(".", "custom_models", CUSTOM_MODEL_NAME),
    "EXPORTED_MODEL": os.path.join(".", "custom_models", "export", CUSTOM_MODEL_NAME),
    "EXPORTED_MODEL_LITE": os.path.join(".", "custom_models", "export", CUSTOM_MODEL_NAME+"_lite"),
}

PATH_DATASET = PATHS["DATASET"]
PATH_MODELS = PATHS["MODELS"]
PATH_PRETRAINED_MODEL = PATHS["PRETRAINED_MODEL"]
PATH_APIMODEL = PATHS["APIMODEL"]
PATH_PROTOC = PATHS["PROTOC"]
PATH_CUSTOM_MODELS = PATHS["CUSTOM_MODELS"]
PATH_CUSTOM_MODEL = PATHS["CUSTOM_MODEL"]
PATH_EXPORTED_MODEL = PATHS["EXPORTED_MODEL"]
PATH_EXPORTED_MODEL_LITE = PATHS["EXPORTED_MODEL_LITE"]
PATH_PIPELINE_CONFIG = os.path.join(PATH_CUSTOM_MODEL, 'pipeline.config')

In [4]:
os. getcwd()

'/home/vitalii/course_project'

In [21]:
for path in PATHS.values():
    if not os.path.exists(path):
        os.makedirs(path)

Download Dataset

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="6vXP2WRz5mhRApbOx39a")
project = rf.workspace("uni-obabi").project("3d_fdm_faults_detection")
dataset = project.version(1).download(model_format="tfrecord",location=PATH_DATASET)

Getting Tensorflow Models and Its Dependencies

In [ ]:
if not os.path.exists(os.path.join(PATH_APIMODEL, 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {PATH_APIMODEL}
    #as far as I use 2.10.1 tensorflow; to know your version tf. __version__
    # !cd {PATH_APIMODEL} && git checkout tags/v2.10.1

In [ ]:
#hz
# !pip install tensorflow-object-detection-api

In [ ]:
#may not work
!sudo apt-get install protobuf-compiler

In [ ]:
!cd {os.path.join(PATH_APIMODEL, 'research')} &&  protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

In [ ]:
VERIFICATION_SCRIPT = os.path.join(PATH_APIMODEL, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [22]:
if not os.path.exists(PATH_PRETRAINED_MODEL+'.tar.gz'):
    !wget {MODEL_URL} -P {PATH_MODELS}
    !cd {PATH_MODELS} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2024-01-22 00:13:11--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.16.27, 172.217.16.59, 216.58.215.91, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.16.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘./models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  10.2MB/s    in 26s     

2024-01-22 00:13:37 (9.12 MB/s) - ‘./models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpo

In [ ]:
import object_detection

In [23]:
print(os.path.join(PATH_PRETRAINED_MODEL, 'pipeline.config'))
print(PATH_CUSTOM_MODEL)

./models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
./custom_models/ssd_resnet50_v1_fpn_640x640


In [24]:
PATH_CONFIG = os.path.join(PATH_PRETRAINED_MODEL, 'pipeline.config')
!cp {PATH_CONFIG} {PATH_CUSTOM_MODEL}
PATH_PIPELINE_CONFIG = os.path.join(PATH_CUSTOM_MODEL, 'pipeline.config')
print(PATH_PIPELINE_CONFIG)

./custom_models/ssd_resnet50_v1_fpn_640x640/pipeline.config


<h5>Edit Training Pipeline<h5>

In [ ]:
!pip list

In [ ]:
pip install protobuf==3.20.3

In [27]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [10]:
config = config_util.get_configs_from_pipeline_file(PATH_PIPELINE_CONFIG)

In [ ]:
config

In [25]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PATH_PIPELINE_CONFIG, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
PATH_DATASET

In [26]:
pipeline_config.model.ssd.num_classes = 5  # Blobs, Cracks, Spaghetti, Stringging, Under Extrusion
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.abspath(os.path.join(PATH_PRETRAINED_MODEL, 'checkpoint', 'ckpt-0'))
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

path_train_label = os.path.abspath(os.path.join(PATH_DATASET, 'train', 'FDM-defects_label_map.pbtxt'))
path_train_data = os.path.abspath(os.path.join(PATH_DATASET, 'train', 'FDM-defects.tfrecord'))
path_eval_label = os.path.abspath(os.path.join(PATH_DATASET, 'valid', 'FDM-defects_label_map.pbtxt'))
path_eval_data = os.path.abspath(os.path.join(PATH_DATASET, 'valid', 'FDM-defects.tfrecord'))

pipeline_config.train_input_reader.label_map_path= path_train_label
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [path_train_data]
pipeline_config.eval_input_reader[0].label_map_path = path_eval_label
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [path_eval_data]

In [28]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(PATH_PIPELINE_CONFIG, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [ ]:
PATH_MODEL

Training

In [41]:
# use --checkpoint_every_n= to define checkpoints frequency
TRAINING_SCRIPT = os.path.join(PATH_APIMODEL, 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=25000".\
            format(TRAINING_SCRIPT, PATH_CUSTOM_MODEL ,PATH_PIPELINE_CONFIG)
print(command)

python ./apimodel/research/object_detection/model_main_tf2.py --model_dir=./custom_models/ssd_mobilenet_fpn_640x640 --pipeline_config_path=./custom_models/ssd_mobilenet_fpn_640x640/pipeline.config --num_train_steps=25000


In [36]:
!{command}

2024-01-22 00:31:16.569835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 00:31:17.350343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-22 00:31:19.518102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-22 00:31:19.538374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-22 00:31:19.538478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

In [35]:
#to run tensorboad run in console
tensorboard_train_command = "tensorboard {}".format(os.path.join(PATH_CUSTOM_MODEL, 'train'))
print(tensorboard_train_command)

tensorboard ./custom_models/ssd_resnet50_v1_fpn_640x640/train


In [42]:
eval_model_command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}" \
                    .format(TRAINING_SCRIPT, PATH_CUSTOM_MODEL,PATH_PIPELINE_CONFIG, PATH_CUSTOM_MODEL)
print(eval_model_command)

python ./apimodel/research/object_detection/model_main_tf2.py --model_dir=./custom_models/ssd_mobilenet_fpn_640x640 --pipeline_config_path=./custom_models/ssd_mobilenet_fpn_640x640/pipeline.config --checkpoint_dir=./custom_models/ssd_mobilenet_fpn_640x640


In [43]:
!{eval_model_command}

2024-01-23 06:02:21.356506: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 06:02:22.749767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-23 06:02:26.081765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 06:02:26.136279: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 06:02:26.136384: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

In [8]:
#to check eval in tensorboad run
tensorboard_eval_command = "tensorboard --logdir {}".format(os.path.join(PATH_CUSTOM_MODEL, 'eval'))
print(tensorboard_eval_command)

tensorboard --logdir ./custom_models/ssd_mobilenet_fpn_640x640/eval


Eport model

In [19]:
PATH_EXPORTED_MODEL

'./custom_models/export/ssd_mobilenet_fpn_640x640'

In [20]:
PATH_EXPORT_SCRIPT = os.path.join(PATH_APIMODEL, 'research', 'object_detection', 'exporter_main_v2.py')
export_command = "python {} --input_type image_tensor --pipeline_config_path {} --trained_checkpoint_dir {} --output_directory {}".\
            format(PATH_EXPORT_SCRIPT, PATH_PIPELINE_CONFIG, PATH_CUSTOM_MODEL, PATH_EXPORTED_MODEL)
print(export_command)

python ./apimodel/research/object_detection/exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./custom_models/ssd_mobilenet_fpn_640x640/pipeline.config --trained_checkpoint_dir ./custom_models/ssd_mobilenet_fpn_640x640 --output_directory ./custom_models/export/ssd_mobilenet_fpn_640x640


In [21]:
!{export_command}

2024-01-20 21:31:06.952092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 21:31:07.733291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-20 21:31:09.499395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 21:31:09.534590: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 21:31:09.534678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

In [27]:
PATH_EXPORT_TF_LITE_SCRIPT = os.path.join(PATH_APIMODEL, 'research', 'object_detection', 'export_tflite_graph_tf2.py')
export_lite_command = "python {} --pipeline_config_path {} --trained_checkpoint_dir {} --output_directory {}".\
            format(PATH_EXPORT_TF_LITE_SCRIPT, PATH_PIPELINE_CONFIG, PATH_CUSTOM_MODEL, PATH_EXPORTED_MODEL_LITE)
print(export_lite_command)

python ./apimodel/research/object_detection/export_tflite_graph_tf2.py --pipeline_config_path ./custom_models/ssd_mobilenet_fpn_640x640/pipeline.config --trained_checkpoint_dir ./custom_models/ssd_mobilenet_fpn_640x640 --output_directory ./custom_models/export/ssd_mobilenet_fpn_640x640_lite


In [28]:
!{export_lite_command}

2024-01-20 21:51:40.900607: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 21:51:41.668372: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-20 21:51:43.537143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 21:51:43.574526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 21:51:43.574646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

In [32]:
PATH_EXPORTED_MODEL_LITE_RES = os.path.join(PATH_EXPORTED_MODEL_LITE, 'saved_model')
PATH_TFLITE_MODEL = os.path.join(PATH_EXPORTED_MODEL_LITE, CUSTOM_MODEL_NAME + '.tflite')

converter = tf.lite.TFLiteConverter.from_saved_model(PATH_EXPORTED_MODEL_LITE_RES)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(PATH_TFLITE_MODEL, 'wb') as f:
  f.write(tflite_model)

2024-01-20 22:04:10.890600: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 22:04:10.922095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 22:04:10.922149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 22:04:10.925354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 22:04:10.925442: I tensorflow/compile